# 06 - Evaluation Comparison: Judging the Patterns

## What is this notebook?

This notebook runs the same test documents through all 5 workflow patterns and compares the results. An LLM judge then assesses which approach is most robust for production privilege review.

## Why compare patterns?

Each pattern has strengths and weaknesses:
- Some are faster but less accurate
- Some catch edge cases but cost more
- Some are better for simple documents, others for complex ones

A systematic comparison helps choose the right pattern for your use case.

## What we'll do

1. Run 3 test documents through all 5 patterns:
   - **Doc A:** Clearly privileged (lawyer-client advice)
   - **Doc B:** Clearly not privileged (business operational)
   - **Doc C:** Edge case (waiver risk)
2. Collect results and metrics (classification, confidence, issues found)
3. Have an LLM judge evaluate the approaches
4. Produce recommendations for production use

## The Patterns

| # | Pattern | Strength |
|---|---------|----------|
| 1 | Prompt Chaining | Auditable fixed steps |
| 2 | Routing | Specialist classifiers |
| 3 | Parallelization | Consensus from multiple models |
| 4 | Orchestrator-Worker | Dynamic breakdown |
| 5 | Evaluator-Optimizer | Self-critique |

## Step 1: Setup

Import libraries and create clients for all patterns.

**What this does:**
- `from openai import OpenAI` — loads the OpenAI library
- `import time` — for measuring execution time
- Creates clients for OpenAI and local Ollama (for parallelization)
- Defines models to use across all patterns

In [ ]:
from openai import OpenAI
from IPython.display import display, Markdown
import time
import pandas as pd
from datetime import datetime

# OpenAI client (cloud)
openai_client = OpenAI()

# Ollama client (local) - for parallelization pattern
ollama_client = OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama"
)

# Models
MODEL_OPENAI = "gpt-4.1-nano"
MODEL_LLAMA = "llama3.2:1b"

print("Clients configured:")
print(f"  OpenAI: {MODEL_OPENAI}")
print(f"  Ollama: {MODEL_LLAMA}")

## Step 2: Create Test Documents

Three documents with different privilege characteristics to test all patterns.

**What this does:**
- **Doc A:** Clearly privileged - straightforward lawyer-client legal advice
- **Doc B:** Clearly not privileged - operational business email, no lawyers
- **Doc C:** Edge case - legal advice forwarded to opposing party (waiver risk)

In [ ]:
# Document A: Clearly privileged
doc_a = {
    "id": "DOC_A",
    "description": "Clearly privileged - lawyer-client advice",
    "expected": "PRIVILEGED",
    "content": """
From: michael.wong@wongpartners.com.au
To: sarah.chen@acmecorp.com.au
Date: 2024-03-15
Subject: Confidential legal advice - BuildRight dispute

Dear Sarah,

Further to our meeting, I provide the following confidential legal advice 
regarding the BuildRight dispute.

In my opinion:
1. The limitation clause in clause 14.3 is enforceable
2. Your maximum exposure is capped at $500,000
3. I recommend we make a without prejudice offer of $350,000

This advice is strictly confidential and subject to legal professional privilege.

Please do not share this advice outside the legal team.

Kind regards,
Michael Wong
Partner, Wong & Partners
"""
}

# Document B: Clearly not privileged
doc_b = {
    "id": "DOC_B",
    "description": "Clearly not privileged - business operational",
    "expected": "NOT_PRIVILEGED",
    "content": """
From: john.smith@acmecorp.com.au
To: accounts@acmecorp.com.au
CC: jane.doe@acmecorp.com.au
Date: 2024-03-14
Subject: Q3 Marketing Budget Approval

Hi Team,

Please find attached the Q3 marketing budget for approval.

Key items:
- Digital advertising: $450,000
- Trade shows: $200,000
- Print materials: $150,000

Total: $800,000

This is within our approved annual budget. Please process for payment.

Thanks,
John Smith
CFO, ACME Corporation
"""
}

# Document C: Edge case - waiver risk
doc_c = {
    "id": "DOC_C",
    "description": "Edge case - waiver risk (advice forwarded to opposing party)",
    "expected": "UNCERTAIN/WAIVER",
    "content": """
From: sarah.chen@acmecorp.com.au
To: michael.wong@wongpartners.com.au
CC: david.wilson@buildright.com.au
Date: 2024-03-18
Subject: FW: Settlement proposal

Michael,

As discussed, I'm forwarding your advice to David Wilson at BuildRight 
to progress settlement discussions.

David - based on our legal advice, we're prepared to offer $350,000 to 
resolve this matter. Michael has advised this represents fair value 
given the limitation clause.

Can we meet next week?

Sarah

--- Original Message ---
From: michael.wong@wongpartners.com.au
To: sarah.chen@acmecorp.com.au
Subject: Confidential advice

Sarah, my confidential advice is to offer $350k based on the limitation clause.
This is privileged and confidential.

Michael
"""
}

test_documents = [doc_a, doc_b, doc_c]

print("Test documents created:")
for doc in test_documents:
    print(f"  {doc['id']}: {doc['description']} (Expected: {doc['expected']})")

## Step 3: Create Simplified Pattern Functions

Compact versions of each workflow pattern for comparison testing.

**What this does:**
- Creates a function for each of the 5 patterns
- Each function takes a document and returns classification, confidence, and key findings
- Simplified versions to enable fair comparison on same documents
- Records execution time for each pattern

In [ ]:
# Pattern 1: Prompt Chaining (simplified to key steps)
def pattern_chaining(document):
    """Prompt chaining: sequential steps"""
    start = time.time()
    
    # Step 1: Identify parties
    r1 = openai_client.chat.completions.create(
        model=MODEL_OPENAI,
        messages=[{"role": "user", "content": f"Identify the sender, recipient and their roles in this document:\n{document['content']}\n\nFormat: SENDER: [name-role] RECIPIENT: [name-role]"}]
    ).choices[0].message.content
    
    # Step 2: Check for lawyer
    r2 = openai_client.chat.completions.create(
        model=MODEL_OPENAI,
        messages=[{"role": "user", "content": f"Based on these parties: {r1}\n\nIs a lawyer involved? Answer: LAWYER_INVOLVED: [Yes/No] REASONING: [brief]"}]
    ).choices[0].message.content
    
    # Step 3: Final determination
    r3 = openai_client.chat.completions.create(
        model=MODEL_OPENAI,
        messages=[{"role": "user", "content": f"""Document:\n{document['content']}\n\nParties: {r1}\nLawyer analysis: {r2}\n\nApply Evidence Act 1995 (Cth) ss 118-119. Provide final privilege classification.\n\nFormat:\nCLASSIFICATION: [PRIVILEGED/NOT_PRIVILEGED/UNCERTAIN]\nCONFIDENCE: [0-100]\nKEY_FINDING: [one sentence]"""}]
    ).choices[0].message.content
    
    elapsed = time.time() - start
    return {"pattern": "Chaining", "result": r3, "time": elapsed, "calls": 3}


# Pattern 2: Routing (simplified)
def pattern_routing(document):
    """Routing: classify type then route to specialist"""
    start = time.time()
    
    # Route: determine type
    route = openai_client.chat.completions.create(
        model=MODEL_OPENAI,
        messages=[{"role": "user", "content": f"Classify this document type (EMAIL/BOARD_MINUTE/FILE_NOTE/OTHER):\n{document['content']}\n\nDOCUMENT_TYPE: [type]"}]
    ).choices[0].message.content
    
    # Specialist classifier
    r2 = openai_client.chat.completions.create(
        model=MODEL_OPENAI,
        messages=[{"role": "user", "content": f"""You are a specialist privilege classifier for {route}.
        
Document:\n{document['content']}\n\nApply Evidence Act 1995 (Cth) ss 118-119.\n\nFormat:\nCLASSIFICATION: [PRIVILEGED/NOT_PRIVILEGED/UNCERTAIN]\nCONFIDENCE: [0-100]\nKEY_FINDING: [one sentence]"""}]
    ).choices[0].message.content
    
    elapsed = time.time() - start
    return {"pattern": "Routing", "result": r2, "time": elapsed, "calls": 2}


# Pattern 3: Parallelization (simplified)
def pattern_parallelization(document):
    """Parallelization: two models, check consensus"""
    start = time.time()
    
    prompt = f"""Analyse for Australian legal professional privilege (Evidence Act 1995 (Cth) ss 118-119):\n{document['content']}\n\nFormat:\nCLASSIFICATION: [PRIVILEGED/NOT_PRIVILEGED/UNCERTAIN]\nCONFIDENCE: [0-100]\nKEY_FINDING: [one sentence]"""
    
    # OpenAI
    r1 = openai_client.chat.completions.create(
        model=MODEL_OPENAI,
        messages=[{"role": "user", "content": prompt}]
    ).choices[0].message.content
    
    # Llama (local)
    r2 = ollama_client.chat.completions.create(
        model=MODEL_LLAMA,
        messages=[{"role": "user", "content": prompt}]
    ).choices[0].message.content
    
    # Check consensus
    class1 = "PRIVILEGED" if "PRIVILEGED" in r1 and "NOT_PRIVILEGED" not in r1 else ("NOT_PRIVILEGED" if "NOT_PRIVILEGED" in r1 else "UNCERTAIN")
    class2 = "PRIVILEGED" if "PRIVILEGED" in r2 and "NOT_PRIVILEGED" not in r2 else ("NOT_PRIVILEGED" if "NOT_PRIVILEGED" in r2 else "UNCERTAIN")
    
    consensus = class1 == class2
    final = class1 if consensus else "UNCERTAIN"
    
    result = f"CLASSIFICATION: {final}\nCONSENSUS: {consensus}\nOPENAI: {class1}\nLLAMA: {class2}\nKEY_FINDING: {'Models agree' if consensus else 'Models disagree - needs review'}"
    
    elapsed = time.time() - start
    return {"pattern": "Parallelization", "result": result, "time": elapsed, "calls": 2}


# Pattern 4: Orchestrator-Worker (simplified)
def pattern_orchestrator(document):
    """Orchestrator-worker: dynamic breakdown"""
    start = time.time()
    
    # Orchestrator identifies subtasks
    r1 = openai_client.chat.completions.create(
        model=MODEL_OPENAI,
        messages=[{"role": "user", "content": f"List 3 key analysis tasks needed for privilege assessment of this document:\n{document['content']}\n\nFormat: TASK1: [task] TASK2: [task] TASK3: [task]"}]
    ).choices[0].message.content
    
    # Worker executes
    r2 = openai_client.chat.completions.create(
        model=MODEL_OPENAI,
        messages=[{"role": "user", "content": f"Execute these analysis tasks on the document:\n\nTasks: {r1}\n\nDocument:\n{document['content']}\n\nProvide findings for each task briefly."}]
    ).choices[0].message.content
    
    # Orchestrator synthesizes
    r3 = openai_client.chat.completions.create(
        model=MODEL_OPENAI,
        messages=[{"role": "user", "content": f"Based on these findings:\n{r2}\n\nProvide final privilege classification under Evidence Act 1995 (Cth) ss 118-119.\n\nFormat:\nCLASSIFICATION: [PRIVILEGED/NOT_PRIVILEGED/UNCERTAIN]\nCONFIDENCE: [0-100]\nKEY_FINDING: [one sentence]"}]
    ).choices[0].message.content
    
    elapsed = time.time() - start
    return {"pattern": "Orchestrator", "result": r3, "time": elapsed, "calls": 3}


# Pattern 5: Evaluator-Optimizer (simplified)
def pattern_evaluator(document):
    """Evaluator-optimizer: generate, critique, improve"""
    start = time.time()
    
    # Generate initial
    r1 = openai_client.chat.completions.create(
        model=MODEL_OPENAI,
        messages=[{"role": "user", "content": f"Classify for privilege under Evidence Act 1995 (Cth) ss 118-119:\n{document['content']}\n\nFormat: CLASSIFICATION: [PRIVILEGED/NOT_PRIVILEGED/UNCERTAIN] REASONING: [brief]"}]
    ).choices[0].message.content
    
    # Evaluate/critique
    r2 = openai_client.chat.completions.create(
        model=MODEL_OPENAI,
        messages=[{"role": "user", "content": f"Critique this privilege assessment. Look for errors, missed waiver issues, third party disclosures:\n\nDocument:\n{document['content']}\n\nAssessment:\n{r1}\n\nFormat: ERRORS_FOUND: [Yes/No] CRITIQUE: [brief]"}]
    ).choices[0].message.content
    
    # Optimize
    r3 = openai_client.chat.completions.create(
        model=MODEL_OPENAI,
        messages=[{"role": "user", "content": f"Provide revised classification based on critique:\n\nOriginal: {r1}\nCritique: {r2}\n\nFormat:\nCLASSIFICATION: [PRIVILEGED/NOT_PRIVILEGED/UNCERTAIN]\nCONFIDENCE: [0-100]\nKEY_FINDING: [one sentence]"}]
    ).choices[0].message.content
    
    elapsed = time.time() - start
    return {"pattern": "Evaluator", "result": r3, "time": elapsed, "calls": 3}


print("Pattern functions created:")
print("  1. pattern_chaining()")
print("  2. pattern_routing()")
print("  3. pattern_parallelization()")
print("  4. pattern_orchestrator()")
print("  5. pattern_evaluator()")

## Step 4: Run All Patterns on All Documents

Execute each pattern on each test document and collect results.

**What this does:**
- Runs all 5 patterns on all 3 documents (15 total classifications)
- Records classification, confidence, time, and LLM calls for each
- Builds a comparison matrix for analysis
- This may take 1-2 minutes to complete

In [ ]:
patterns = [
    ("Chaining", pattern_chaining),
    ("Routing", pattern_routing),
    ("Parallelization", pattern_parallelization),
    ("Orchestrator", pattern_orchestrator),
    ("Evaluator", pattern_evaluator)
]

all_results = []

print("Running all patterns on all documents...\n")

for doc in test_documents:
    print(f"Processing {doc['id']}: {doc['description']}")
    
    for pattern_name, pattern_func in patterns:
        print(f"  Running {pattern_name}...", end=" ")
        try:
            result = pattern_func(doc)
            
            # Extract classification from result
            classification = "UNKNOWN"
            confidence = "0"
            for line in result['result'].split('\n'):
                if line.startswith("CLASSIFICATION:"):
                    classification = line.split(':')[1].strip()
                if line.startswith("CONFIDENCE:"):
                    confidence = line.split(':')[1].strip()
            
            all_results.append({
                "doc_id": doc['id'],
                "expected": doc['expected'],
                "pattern": pattern_name,
                "classification": classification,
                "confidence": confidence,
                "time_seconds": round(result['time'], 2),
                "llm_calls": result['calls'],
                "raw_result": result['result']
            })
            print(f"✓ ({result['time']:.1f}s)")
        except Exception as e:
            print(f"✗ Error: {e}")
            all_results.append({
                "doc_id": doc['id'],
                "expected": doc['expected'],
                "pattern": pattern_name,
                "classification": "ERROR",
                "confidence": "0",
                "time_seconds": 0,
                "llm_calls": 0,
                "raw_result": str(e)
            })
    print()

print("All patterns complete!")

## Step 5: Display Results Comparison Table

View the results of all patterns across all documents.

**What this does:**
- Creates a comparison table showing all results
- Highlights which patterns matched expected outcomes
- Shows timing and cost (LLM calls) for each pattern
- Identifies which patterns caught the edge case correctly

In [ ]:
# Create DataFrame
df = pd.DataFrame(all_results)

# Display summary table
display(Markdown("### Results by Document"))

for doc_id in df['doc_id'].unique():
    doc_df = df[df['doc_id'] == doc_id]
    expected = doc_df['expected'].iloc[0]
    
    display(Markdown(f"\n**{doc_id}** (Expected: {expected})"))
    display(doc_df[['pattern', 'classification', 'confidence', 'time_seconds', 'llm_calls']])

# Summary statistics
display(Markdown("### Summary Statistics"))

summary = df.groupby('pattern').agg({
    'time_seconds': 'mean',
    'llm_calls': 'first'
}).round(2)
summary.columns = ['Avg Time (s)', 'LLM Calls']
display(summary)

# Accuracy check
display(Markdown("### Accuracy Check"))

def check_correct(row):
    if row['expected'] == 'PRIVILEGED' and 'PRIVILEGED' in row['classification'] and 'NOT' not in row['classification']:
        return '✓'
    elif row['expected'] == 'NOT_PRIVILEGED' and 'NOT_PRIVILEGED' in row['classification']:
        return '✓'
    elif row['expected'] == 'UNCERTAIN/WAIVER' and ('UNCERTAIN' in row['classification'] or 'WAIVER' in row['classification'] or 'PARTIAL' in row['classification']):
        return '✓'
    else:
        return '✗'

df['correct'] = df.apply(check_correct, axis=1)

accuracy_table = df.pivot(index='pattern', columns='doc_id', values='correct')
display(accuracy_table)

# Count correct per pattern
correct_counts = df.groupby('pattern')['correct'].apply(lambda x: (x == '✓').sum())
display(Markdown(f"\n**Correct classifications out of 3:**"))
for pattern, count in correct_counts.items():
    display(Markdown(f"- {pattern}: {count}/3"))

## Step 6: LLM Judge Evaluation

Have an LLM judge assess which pattern is most robust for production privilege review.

**What this does:**
- Presents all results to a senior "judge" LLM
- Asks for assessment of each pattern's strengths and weaknesses
- Requests recommendation for production use
- Considers accuracy, speed, cost, and ability to catch edge cases

In [ ]:
# Prepare results summary for judge
results_summary = """
PATTERN COMPARISON RESULTS - Australian Legal Professional Privilege Classification

TEST DOCUMENTS:
- DOC_A: Clearly privileged (lawyer-client advice) - Expected: PRIVILEGED
- DOC_B: Clearly not privileged (business operational) - Expected: NOT_PRIVILEGED  
- DOC_C: Edge case with waiver risk (advice forwarded to opposing party) - Expected: UNCERTAIN/WAIVER

RESULTS:

Pattern: CHAINING (3 LLM calls, avg 2.14s)
- DOC_A: PRIVILEGED (95%) ✓
- DOC_B: NOT_PRIVILEGED (100%) ✓
- DOC_C: PRIVILEGED (95%) ✗ MISSED WAIVER

Pattern: ROUTING (2 LLM calls, avg 1.26s)
- DOC_A: PRIVILEGED (95%) ✓
- DOC_B: NOT_PRIVILEGED (85%) ✓
- DOC_C: PRIVILEGED (95%) ✗ MISSED WAIVER

Pattern: PARALLELIZATION (2 LLM calls, avg 7.90s)
- DOC_A: UNCERTAIN ✗ (models disagreed)
- DOC_B: UNCERTAIN ✗ (models disagreed)
- DOC_C: UNCERTAIN ✓ (correctly flagged for review)

Pattern: ORCHESTRATOR-WORKER (3 LLM calls, avg 4.38s)
- DOC_A: PRIVILEGED (95%) ✓
- DOC_B: PRIVILEGED (85%) ✗ WRONG
- DOC_C: PRIVILEGED (95%) ✗ MISSED WAIVER

Pattern: EVALUATOR-OPTIMIZER (3 LLM calls, avg 4.79s)
- DOC_A: PRIVILEGED (85%) ✓
- DOC_B: NOT_PRIVILEGED (85%) ✓
- DOC_C: UNCERTAIN (65%) ✓ CAUGHT WAIVER RISK
"""

judge_prompt = f"""
You are a senior Australian litigation partner evaluating AI privilege classification systems for production use.

{results_summary}

Provide your expert assessment:

1. ACCURACY ANALYSIS: Which patterns performed best/worst and why?

2. EDGE CASE HANDLING: Which patterns correctly identified the waiver risk in DOC_C? Why did others miss it?

3. COST-BENEFIT ANALYSIS: Considering speed, LLM calls (cost), and accuracy - which offers best value?

4. RISK ASSESSMENT: For legal privilege review where false negatives (missing privilege) could waive privilege permanently, which pattern is safest?

5. PRODUCTION RECOMMENDATION: Which pattern(s) would you recommend for:
   a) High-volume first-pass review (thousands of documents)
   b) High-stakes final review (documents going to court)
   c) Edge cases flagged for human review

6. FINAL VERDICT: Your overall recommendation with reasoning.

Format your response with clear headings for each section.
"""

print("Judge evaluating patterns...")
judge_response = openai_client.chat.completions.create(
    model=MODEL_OPENAI,
    messages=[{"role": "user", "content": judge_prompt}]
)

judge_result = judge_response.choices[0].message.content
display(Markdown(f"## LLM Judge Assessment\n\n{judge_result}"))

## Step 7: Export to CSV for Senior Lawyer Review

Create a comprehensive CSV with all comparison results.

**What this does:**
- Exports all 15 classification results to CSV
- Includes pattern, classification, confidence, timing, and accuracy
- Provides a complete audit trail of the comparison
- Senior lawyers can review which patterns to deploy in production

In [ ]:
# Add the correct column to the dataframe
df['correct'] = df.apply(check_correct, axis=1)

# Select columns for export
export_df = df[['doc_id', 'expected', 'pattern', 'classification', 'confidence', 'time_seconds', 'llm_calls', 'correct']]

# Export to CSV
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
csv_filename = f"privilege_review_comparison_{timestamp}.csv"
export_df.to_csv(csv_filename, index=False)

# Display the full table
display(Markdown("### Complete Results Table"))
display(export_df)

display(Markdown(f"**Exported:** `{csv_filename}`"))

# Summary
display(Markdown("""
### Summary

| Pattern | Accuracy | Avg Time | Best For |
|---------|----------|----------|----------|
| Evaluator | 3/3 (100%) | 4.79s | High-stakes review, edge cases |
| Chaining | 2/3 (67%) | 2.14s | Auditable sequential analysis |
| Routing | 2/3 (67%) | 1.26s | High-volume first-pass |
| Orchestrator | 1/3 (33%) | 4.38s | Complex multi-part documents |
| Parallelization | 1/3 (33%) | 7.90s | Consensus checking |

**Recommendation:** Hybrid approach using Routing for bulk screening and Evaluator for final review.
"""))

## Conclusion: Pattern Comparison for LPP Classification

### What We Built

A systematic comparison of all 5 workflow patterns against the same test documents:

| Document | Description | Expected | Challenge |
|----------|-------------|----------|-----------|
| DOC_A | Lawyer-client advice | PRIVILEGED | Straightforward |
| DOC_B | Business operational | NOT_PRIVILEGED | No lawyers involved |
| DOC_C | Advice forwarded to opponent | UNCERTAIN/WAIVER | Subtle waiver risk |

### Key Findings

**Accuracy Results:**
- **Evaluator-Optimizer: 3/3** - Only pattern to catch all cases including waiver
- Chaining: 2/3 - Missed waiver edge case
- Routing: 2/3 - Fast but missed waiver
- Orchestrator: 1/3 - Struggled with clear-cut cases
- Parallelization: 1/3 - Too conservative, defaulted to UNCERTAIN

**The Critical Test (DOC_C - Waiver Risk):**

Only Evaluator-Optimizer correctly identified that forwarding legal advice to the opposing party created a waiver risk. All other patterns confidently (and wrongly) classified it as PRIVILEGED.

This demonstrates why self-critique matters for privilege review.

### Production Recommendations

| Use Case | Pattern | Reason |
|----------|---------|--------|
| **High-volume screening** | Routing | Fastest (1.26s), cheapest (2 calls), accurate on clear cases |
| **Final review** | Evaluator | Catches edge cases, self-corrects errors |
| **Flagging uncertainty** | Parallelization | Model disagreement triggers human review |

### Recommended Hybrid Workflow
```
Incoming Documents
        ↓
   [ROUTING] ← Fast first-pass
        ↓
   ┌────┴────┐
   ↓         ↓
CLEAR     UNCERTAIN
   ↓         ↓
 Done    [EVALUATOR] ← Self-critique
             ↓
        ┌────┴────┐
        ↓         ↓
    RESOLVED   STILL UNCERTAIN
        ↓         ↓
      Done    HUMAN REVIEW
```

### Lessons Learned

1. **No single pattern is best for everything** - match pattern to use case
2. **Edge cases need self-critique** - simple patterns miss subtle issues
3. **Speed vs accuracy trade-off** - Routing is 4x faster but misses nuance
4. **Waiver is hard to catch** - forwarding to third parties needs explicit checking
5. **Hybrid approaches win** - combine patterns for best results

### Complete Notebook Series

| Notebook | Pattern | Key Strength |
|----------|---------|--------------|
| 01 | Prompt Chaining | Auditable fixed steps |
| 02 | Routing | Specialist classifiers by type |
| 03 | Parallelization | Multi-model consensus |
| 04 | Orchestrator-Worker | Dynamic task breakdown |
| 05 | Evaluator-Optimizer | Self-critique catches errors |
| 06 | Comparison | Systematic evaluation |

### Next Steps

1. Expand test set with more edge cases
2. Fine-tune prompts based on error patterns
3. Implement hybrid workflow in production
4. Add human feedback loop for continuous improvement
5. Consider adding Australian case law citations to prompts